<a href="https://colab.research.google.com/github/Sujith2k/An-Aid-for-the-Blind/blob/master/ML/Currency_model/currency_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd gdrive

/content/gdrive


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import random
import os
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array , load_img
from keras.utils import to_categorical
import pandas as ps 
import cv2
from sklearn.model_selection import train_test_split

import keras.backend as K
K.set_learning_phase(0)

In [ ]:
labels = ['10','20','50','100','200','500','2000']
train_imgs_names= os.listdir("My Drive/cohort/Dataset/Currency_Train")
len(train_imgs_names)

584

In [ ]:
random.shuffle(train_imgs_names)
train_imgs , pre_train_labels = [],[]

for img in train_imgs_names:
    train_imgs.append( cv2.resize( cv2.imread("My Drive/cohort/Dataset/Currency_Train/"+img) , (150,150) ,interpolation = cv2.INTER_CUBIC ) )

    for index,label in enumerate(labels):
        if(label + '_' in img):
            pre_train_labels.append(index)

In [ ]:
print(pre_train_labels[0])
train_labels = to_categorical(pre_train_labels)
train_labels[0]

4


array([0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3) ,activation = 'relu' , input_shape =(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64 , (3,3) , activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128 , (3,3) , activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128 , (3,3) , activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation = 'relu'))
model.add(layers.Dense(7, activation = 'sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)      

In [ ]:
model.compile(loss='binary_crossentropy',optimizer = optimizers.RMSprop(lr = 1e-4),metrics= ['acc'])

In [ ]:
x_train , x_val , y_train , y_val = train_test_split( train_imgs , train_labels , test_size = 0.2 )

x_train = np.asarray(x_train)
x_val = np.asarray(x_val)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255 , rotation_range = 40 , width_shift_range = 0.2 , height_shift_range = 0.2 , shear_range = 0.2 , zoom_range = 0.2 , horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow(x_train, y_train , batch_size = 32)
val_generator = val_datagen.flow(x_val, y_val , batch_size = 32)

In [ ]:
model.fit_generator(train_generator , steps_per_epoch = 100 , epochs = 50 , validation_data = val_generator , validation_steps = 100)

Epoch 1/50
100/100 [==============================] - 25s 250ms/step - loss: 0.3920 - acc: 0.8520 - val_loss: 0.3431 - val_acc: 0.8535
Epoch 2/50
100/100 [==============================] - 18s 180ms/step - loss: 0.3001 - acc: 0.8740 - val_loss: 0.3380 - val_acc: 0.8852
Epoch 3/50
100/100 [==============================] - 17s 175ms/step - loss: 0.2593 - acc: 0.8922 - val_loss: 0.2130 - val_acc: 0.8913
Epoch 4/50
100/100 [==============================] - 17s 172ms/step - loss: 0.2336 - acc: 0.9017 - val_loss: 0.2510 - val_acc: 0.8987
Epoch 5/50
100/100 [==============================] - 18s 183ms/step - loss: 0.2189 - acc: 0.9073 - val_loss: 0.2035 - val_acc: 0.9219
Epoch 6/50
100/100 [==============================] - 17s 175ms/step - loss: 0.2060 - acc: 0.9137 - val_loss: 0.1699 - val_acc: 0.9231
Epoch 7/50
100/100 [==============================] - 17s 172ms/step - loss: 0.1917 - acc: 0.9185 - val_loss: 0.1955 - val_acc: 0.9267
Epoch 8/50
100/100 [==============================] - 1

In [ ]:
model.save('My Drive/cohort/currency.h5')